In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from IPython.display import display

print("가설2: 디커플링 → 신용경색 Granger 시작")

# ===============================
# 1. 디커플링 로드 + 월 평균 처리
# ===============================
df_dec = pd.read_csv("../../윤성/data/decoupling.csv")

df_dec["date"] = pd.to_datetime(df_dec["date"])
df_dec = df_dec.groupby("date", as_index=False)["gap"].mean()
df_dec = df_dec.sort_values("date").reset_index(drop=True)

print("디커플링 데이터 확인")
display(df_dec.head())

# ===============================
# 2. 신용스프레드 로드
# ===============================
df_sp = pd.read_csv("../data/전처리_신용스프레드.csv")

# 날짜 컬럼 자동 탐색
date_col = [c for c in df_sp.columns if "date" in c.lower() or "기간" in c][0]
df_sp = df_sp.rename(columns={date_col:"date"})
df_sp["date"] = pd.to_datetime(df_sp["date"])

# 스프레드 컬럼 자동 탐색
val_col = [c for c in df_sp.columns if c!="date"][0]
df_sp = df_sp[["date", val_col]].rename(columns={val_col:"SPREAD"})
df_sp = df_sp.sort_values("date").reset_index(drop=True)

print("스프레드 데이터 확인")
display(df_sp.head())

# ===============================
# 3. 병합 (월 기준 유지)
# ===============================
df = df_dec.merge(df_sp, on="date", how="inner")
df = df.sort_values("date").reset_index(drop=True)

print("병합 결과")
display(df.head())

# ===============================
# 4. 정상성 확보 (1차 차분)
# ===============================
df["DECOUPLING_d1"] = df["gap"].diff()
df["SPREAD_d1"] = df["SPREAD"].diff()

df = df.dropna().reset_index(drop=True)

print("차분 후 데이터")
display(df.head())

# ===============================
# 5. Granger 인과검정
# ===============================
def run_granger(df, cause, effect, maxlag=6):
    data = df[[effect, cause]].dropna()
    print("\n" + "="*60)
    print(f"{cause} → {effect} (maxlag={maxlag})")
    print("="*60)

    res = grangercausalitytests(data, maxlag=maxlag, verbose=False)
    pvals = {lag: res[lag][0]["ssr_ftest"][1] for lag in res.keys()}

    out = pd.DataFrame({
        "lag": list(pvals.keys()),
        "p_value": list(pvals.values())
    }).sort_values("lag")

    display(out)

    best_lag = out.loc[out["p_value"].idxmin(),"lag"]
    best_p = out["p_value"].min()

    print(f"최소 p-value: {best_p:.4f} (lag{int(best_lag)})")

print("디커플링 → 스프레드")
run_granger(df, "DECOUPLING_d1", "SPREAD_d1", maxlag=6)

print("\n가설2 Granger 완료")


가설2: 디커플링 → 신용경색 Granger 시작
디커플링 데이터 확인


,date,gap
0,2005-01-01,1.745656
1,2005-02-01,1.364802
2,2005-03-01,0.846905
3,2005-04-01,1.296759
4,2005-05-01,0.773001


스프레드 데이터 확인


,date,SPREAD
0,2004-01-01,0.79
1,2004-02-01,0.80
2,2004-03-01,0.79
3,2004-04-01,0.75
4,2004-05-01,0.68


병합 결과


...

,date,gap,SPREAD,DECOUPLING_d1,SPREAD_d1
0,2005-02-01,1.364802,0.44,-0.380854,-2.000000e-02
1,2005-03-01,0.846905,0.44,-0.517897,8.881784e-16
2,2005-04-01,1.296759,0.42,0.449855,-2.000000e-02
3,2005-05-01,0.773001,0.41,-0.523758,-1.000000e-02
4,2005-06-01,0.502169,0.39,-0.270832,-2.000000e-02


디커플링 → 스프레드

DECOUPLING_d1 → SPREAD_d1 (maxlag=6)


c:\anaconda\envs\data\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


,lag,p_value
0,1,0.023374
1,2,0.038510
2,3,0.021825
3,4,0.045772
4,5,0.038892
5,6,0.065490


최소 p-value: 0.0218 (lag3)

가설2 Granger 완료


### 📌 가설 2: 디커플링 → 신용경색 신호 (Granger 인과검정 결과)

#### 1️⃣ 분석 목적

본 분석은 디커플링 지수의 변화가 이후 신용스프레드 확대를  
선행적으로 설명하는지를 검정하기 위해 수행되었다.

즉,

> 디커플링 지수가 상승하면 이후 신용스프레드가 확대되는가?

를 Granger 인과검정을 통해 확인하였다.

---

#### 2️⃣ 분석 방법

- 데이터: 월 단위  
- 사용 변수:  
  - `DECOUPLING_d1` (디커플링 지수 1차 차분)  
  - `SPREAD_d1` (AA- 회사채 스프레드 1차 차분)  
- 최대 시차: 6개월  
- 검정 방법: **Granger Causality Test (ssr F-test 기반)**  

---

#### 3️⃣ 검정 결과

| lag | p-value |
|-----|----------|
| 1 | 0.0234 |
| 2 | 0.0385 |
| 3 | 0.0218 |
| 4 | 0.0458 |
| 5 | 0.0389 |
| 6 | 0.0655 |

최소 p-value: **0.0218 (lag 3)**

---

#### 4️⃣ 통계적 판단

유의수준 5% 기준에서:

- lag 1 ~ lag 5는 `p < 0.05`
- lag 6은 유의하지 않음 (`p = 0.0655`)

따라서,

> 디커플링 지수의 변화는 이후 신용스프레드 변화를 Granger-cause 한다.

귀무가설(H0):

> 디커플링은 신용스프레드를 예측하지 못한다

는 **기각된다.**

이전 결과보다 p-value 수준은 다소 완화되었으나,  
여전히 1~5개월 시차 구간에서 통계적으로 유의한 인과관계가 확인된다.

---

#### 5️⃣ 경제적 해석

이 결과는 다음을 의미한다.

- 금융–실물 괴리가 확대될 경우  
- 약 1~5개월 이후  
- 신용스프레드가 유의하게 확대되는 경향이 있다.

이는 다음의 논리와 일치한다:

> 실물 둔화가 즉시 신용위험에 반영되기보다는  
> 일정 시차를 두고 기업 부도위험이 재평가되며  
> 신용 프리미엄이 상승한다.

특히 최소 p-value가 lag 3에서 나타났다는 점은  
신용시장 반응이 약 2~3개월의 지연을 두고 나타날 가능성을 시사한다.

즉, 디커플링은 단순한 괴리 지표가 아니라

> **신용경색의 선행 신호 (Early Warning Signal)** 로 기능할 가능성이 높다.

---

#### 6️⃣ 1번 가설과의 연결

1번 가설에서 금융–실물 괴리 구조를 확인하였다면,

2번 가설에서는 그 괴리가

> 실제 금융시장 위험지표(신용스프레드)로 전이되는 동태적 경로

를 통계적으로 입증한 것이다.

즉,

> 금융–실물 괴리 → 신용위험 확대

라는 전이 구조가 실증적으로 확인되었다.

---

### 🎯 최종 결론

디커플링 지수의 증가는 통계적으로 유의하게  
이후 신용스프레드 확대를 선행한다.

비록 이전 분석보다 유의수준은 다소 완화되었으나,  
1~5개월 시차 구간에서 일관된 유의성이 확인되었다는 점에서

> **디커플링은 신용경색의 조기 경고 신호로 활용될 수 있다.**
